In [1]:
%matplotlib inline
import os
import math 
import numpy as np
import pandas as pd 
import seaborn as sns
import random
import matplotlib.pyplot as plt
import statsmodels.formula.api as smf
from scipy.stats import pearsonr, spearmanr, gaussian_kde
from sklearn.decomposition import TruncatedSVD, PCA
from matplotlib.colors import rgb2hex
pd.options.display.max_rows = 2000
pd.options.display.max_columns = 999

sns.set_style('white')
ahba_dir     = '/scr/malta3/workspace/project_iron/AHBA'
ahba_dir     = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSMv3/SEPT10/AHBA/'
ahba_dir2     = '/Users/kanaan/Google Drive/TS-EUROTRAIN/RESULTS_QSMv3/AUG27/AHBA/'
save_fig_dir = '/Users/kanaan/Google Drive/TS-EUROTRAIN/Papers/2016_QSM_paper/Figures_python_v2'

drop_strings = ['coords_native', 'donor_names', 'struct_id', 
                'struct_name', 'top_struct', 'Mean', 'Median', 
                'PC1', 'PC2','PC3', #'SVD1g', 'SVD2g','SVD3g', 'SVD3p', 'SVD2p','SVD3p', 
                'corrected_mni_x', 'corrected_mni_y', 'corrected_mni_z']

import warnings
warnings.filterwarnings('ignore')

# quality control
# drop wells outside the striatal masks 
drop_wells = [2371, 13913, 2379, 127687508]


# READ QSM statmap data 

In [2]:
#df_MNI = pd.read_csv(os.path.join(ahba_dir,'QSM_TSTATS/NIFTI_VALUES_permute_10K_SEPT20.csv'), index_col = 0 )
df_MNI = pd.read_csv(os.path.join(ahba_dir,'QSM_TSTATS/MNI_NIFTI_VALUES_permute_10K_OCT2.csv'), index_col = 0 )

# READ AHBA data (app. 15000 genes)

In [31]:
def return_expression_df(geneset):
    return pd.read_csv(os.path.join(ahba_dir, 'AHBA_%s.csv'%geneset),index_col=0)

# Concat ahba genes 
ahba_a = return_expression_df('18589_a').drop(drop_strings,axis =1)
ahba_b = return_expression_df('18589_b').drop(drop_strings,axis =1)
ahba_c = return_expression_df('18589_c').drop(drop_strings,axis =1)
ahba_d = return_expression_df('18589_d').drop(drop_strings,axis =1)
ahba_e = return_expression_df('18589_e').drop(drop_strings,axis =1)
ahba_f = return_expression_df('18589_f').drop(drop_strings,axis =1)
ahba_g = return_expression_df('18589_g')
AHBA   = pd.concat([ahba_a, ahba_b, ahba_c, ahba_d, ahba_e, ahba_f, ahba_g], axis =1)
#AHBA   = AHBA#.rename(columns={"struct_name": "struct"}) # Dont change struct name here.. this causes indexing problems
AHBA = AHBA.loc[:,~AHBA.columns.duplicated()]# drops duplicate columns

GENE_LIST = sorted(AHBA.columns[:-13])
print 'Total Number of AHBA Genes =', len(GENE_LIST)

#set([x for x in GENE_LIST if GENE_LIST.count(x) > 1])  # checkif there are duplicated columns 

Total Number of AHBA Genes = 14983


In [32]:
AHBA = pd.read_csv(os.path.join(ahba_dir, 'ahba_data', 'AHBA_20737.csv'), index_col = 0)
AHBA = AHBA.iloc[:,:-13]
GENE_LIST = AHBA.columns
print 'Total Number of AHBA Genes =', len(GENE_LIST)


Total Number of AHBA Genes = 20737


In [33]:
def make_chi_gene_correleation(df_ahba, df_chi, roi):
    
    
    df = pd.DataFrame(index=GENE_LIST, columns=['r_val', 'p_val'])    
    
    for gene in GENE_LIST:
        if df_ahba[gene] is not None:
            dfx = pd.DataFrame(index=df_ahba.index)
            dfx[gene]  = df_ahba[gene] 
            dfx[roi]   = df_chi[roi]
            dfx = dfx.drop(drop_wells,axis=0).dropna()

            r_val , p_val= pearsonr(dfx[gene], dfx[roi])
            #print gene, r_val, p_val

            df.loc[gene]['r_val'] = float(r_val)
            df.loc[gene]['p_val'] = float(p_val) 
        
    return df.sort_values('r_val', ascending=0)
        

In [34]:
corr_str3m = make_chi_gene_correleation(AHBA, df_MNI, 'STR3_MOTOR_tstat_CP_2mm')

In [35]:
corr_str = make_chi_gene_correleation(AHBA, df_MNI, 'STR_tstat_CP_2mm')

In [26]:
corr_str_av = make_chi_gene_correleation(AHBA, df_MNI, 'MEAN_STR_ALL_2mm')

In [32]:
corr_gm_av = make_chi_gene_correleation(AHBA, df_MNI, 'MEAN_ALL_GM_2')

In [28]:
corr_pal_av = make_chi_gene_correleation(AHBA, df_MNI, 'MEAN_ALL_PALL_3')


In [36]:
for i in corr_str[0:100].index:
    print i 

RPL23
ZNFX1-AS1
SPATA7
RPL36
HHLA3
RPL39
STARD5
SHB
C9orf123
CIDEA
C12orf43
NDUFA1
NME2P1
SNRPE
PPDPF
SNHG1
EPB41L4A
RPL10A
ZMAT2
ZDHHC11
LOC100287260
CEP63
RPL37
AC005840.2
CACNG7
UCRC
RPS21
NME2
UQCRB
MAGI2
SDCCAG8
SOHLH1
CFL1P1
RPS18
POLR2I
CRYBB2
DDX6
MRPL33
RPL12
ZNF557
C14orf156
BRK1
TTLL13
PFDN5
JOSD2
NACA
GUK1
COX7C
CDON
LY6E
SLA
AGPHD1
XAB2
SERF1B
NDUFV3
RPL29
GOSR2
METTL2B
RPL27
LOC730144
RPL8
MGRN1
LOC648771
LENG1
TBXAS1
CALU
YIF1B
SERF1A
TOMM40L
CRYBB2P1
ZFR
UQCRBP1
UQCR11
COL16A1
LOC100293763
RHCE
C9orf16
RPS13P2
CCL4
PURB
ZNF574
CHCHD2
USMG5
SUCNR1
GINS4
GARNL4
TMEM14B
PPP2R2D
MTRR
RPS27AP11
ACTB
RPL22
RPS7
BAX
RHOBTB1
ARPC2
WDR45
RPL31P11
MYH9
CSTB


In [28]:
corr_str_av[0:200]

,r_val,p_val
C1orf125,0.350465,0.000107167
RASGRP4,0.338666,0.000187792
COL21A1,0.313265,0.000583686
MSTN,0.313206,0.000585164
LOC100133287,0.30731,0.000750828
HOXC10,0.304276,0.000851888
TGM4,0.304146,0.000856481
ATF7,0.302287,0.00092474
CXorf66,0.301101,0.000970836
LOC653111,0.299698,0.0010281
